In [1]:
import boto3
from botocore.config import Config
import sys
import os
import time
import requests
from bs4 import BeautifulSoup


In [2]:
ohio_region  = Config(region_name="us-east-2")
north_virginia_region  = Config(region_name="us-east-1")

In [3]:
ohio_ec2 = boto3.resource('ec2', config=ohio_region)
north_virginia_ec2 = boto3.resource('ec2', config=north_virginia_region)

In [5]:
script_mysql = '''#!/bin/bash
sudo apt update
sudo apt install mysql-server -y
sudo mysql -uroot -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'cloud';CREATE USER 'root'@'%' IDENTIFIED BY 'cloud';GRANT ALL PRIVILEGES ON *.* TO 'root'@'%';FLUSH PRIVILEGES;CREATE DATABASE tasks;"
sudo sed -i 's/bind-address/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo sed -i 's/mysqlx/#banana/g' /etc/mysql/mysql.conf.d/mysqld.cnf
sudo ufw allow 3306/tcp
sudo systemctl restart mysql
'''

In [8]:
security_group = ohio_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup7',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)

security_group2 = north_virginia_ec2.create_security_group(
    Description='string',
    GroupName='SecurityGroup7',
    TagSpecifications=[
        {
            'ResourceType': 'security-group',
            'Tags': [
                {
                    'Key': 'Name',
                    'Value': 'mysql'
                },
            ]
        },
    ],
)




security_group2.authorize_ingress(IpProtocol="tcp",CidrIp="0.0.0.0/0",FromPort=0,ToPort=10000)

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-0714306fbd1f63d60',
   'GroupId': 'sg-0a801d262b45d0122',
   'GroupOwnerId': '903616414837',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 0,
   'ToPort': 10000,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': 'b1261a33-59f1-404c-85df-0e9b206cb952',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b1261a33-59f1-404c-85df-0e9b206cb952',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '721',
   'date': 'Tue, 23 Nov 2021 16:38:56 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [12]:
instance_ohio = ohio_ec2.create_instances(
        ImageId="ami-00399ec92321828f5",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group.group_id],
        UserData = script_mysql
    )



In [11]:
instance_ohio[0].wait_until_running()
instance_ohio[0].reload()
public_ip_ohio = instance_ohio[0].public_ip_address
time.sleep(60)

In [15]:
script_django_mysql = f"""#!/bin/bash
cd /home/ubuntu
sudo apt update
sudo apt-get install python3-dev default-libmysqlclient-dev build-essential -y
git clone https://github.com/raulikeda/tasks.git
cd tasks
sudo sed -i 's/node1/{public_ip_ohio}/g' portfolio/settings.py
sudo sed -i 's/django.db.backends.postgresql/django.db.backends.mysql/g' portfolio/settings.py
sudo sed -i 's/5432/3306/g' portfolio/settings.py
sudo sed -i "s/'USER': 'cloud'/'USER': 'root'/g" portfolio/settings.py
echo "mysqlclient" >> requirements.txt
./install.sh
sudo reboot 
"""

In [16]:
north_virginia = north_virginia_ec2.create_instances(
        ImageId="ami-09e67e426f25ce0d7",
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="jpgianfaldoni",
        SecurityGroupIds = [security_group2.group_id],
        UserData = script_django_mysql
    )

In [17]:
north_virginia[0].wait_until_running()
north_virginia[0].reload()
time.sleep(180)

In [18]:
client_django = requests.session()
client_django.get('http://18.234.150.64:8080/tasks/') 

<Response [200]>

In [20]:
django_img = north_virginia[0].create_image(Name='django_img3')


In [22]:
while django_img.state != 'available':
    django_img.reload()
north_virginia[0].terminate()

{'TerminatingInstances': [{'CurrentState': {'Code': 32,
    'Name': 'shutting-down'},
   'InstanceId': 'i-03dedf60ad95ba82b',
   'PreviousState': {'Code': 16, 'Name': 'running'}}],
 'ResponseMetadata': {'RequestId': '7da0dcc1-3ac9-44a4-a6b6-d2b7d6b9c65c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7da0dcc1-3ac9-44a4-a6b6-d2b7d6b9c65c',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Tue, 23 Nov 2021 16:51:38 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [24]:
autoscaling  = boto3.client('autoscaling', config=north_virginia_region)
elb = boto3.client('elbv2', config=north_virginia_region)


In [25]:
create_launch_response = autoscaling.create_launch_configuration(
    LaunchConfigurationName='LCV999',
    ImageId= django_img.id,
    KeyName='jpgianfaldoni',
    SecurityGroups=[
        security_group2.id,
    ],
    BlockDeviceMappings=[
                        {
                            'DeviceName': '/dev/sda1',
                            'Ebs': {
                                'DeleteOnTermination': True,
                                'VolumeSize': 8,
                                'VolumeType': 'gp2'
                            },
                        },
                ],
    InstanceType='t2.micro',
    
)


In [26]:
create_autoscaling_response = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName='AutoScalingV1',
    LaunchConfigurationName='LCV999',
    MinSize=1,
    MaxSize=3,
    DesiredCapacity=3,
    DefaultCooldown=60,
    AvailabilityZones=[
        'us-east-1a',
        'us-east-1b',
        'us-east-1c',
        'us-east-1d',
        'us-east-1e',
        'us-east-1f',
    ]
)

In [27]:
create_target_response = elb.create_target_group(
    Name='Tgroup5',
    Protocol='HTTP',
    Port=8080,
    VpcId='vpc-ebd11591',
    HealthCheckPath = '/tasks/',
    HealthCheckProtocol='HTTP',
    HealthCheckEnabled=True,
    HealthCheckIntervalSeconds=30,
    HealthCheckTimeoutSeconds=15,
    HealthyThresholdCount=5,
    UnhealthyThresholdCount=5,
    TargetType='instance',
    IpAddressType='ipv4'
)

In [28]:
tg_arn = create_target_response['TargetGroups'][0]['TargetGroupArn']

In [29]:
create_lb_response = elb.create_load_balancer(
    Name='Elb3',
    Subnets=[
        'subnet-54296d1e',
        'subnet-7ebe0d50',
        'subnet-13fa4b4f',
        'subnet-e0f44387',
    ],

    SecurityGroups=[
        security_group2.id,
    ],
    Type='application',
    IpAddressType='ipv4',
)

In [30]:
lb_arn = create_lb_response['LoadBalancers'][0]['LoadBalancerArn']

In [31]:
ab_response = autoscaling.attach_load_balancer_target_groups(
    AutoScalingGroupName='AutoScalingV1',
    TargetGroupARNs=[
        tg_arn,
    ]
)

In [32]:
listener_response = elb.create_listener(
    LoadBalancerArn= lb_arn,
    DefaultActions=[
        {
            'Type': 'forward',
            'TargetGroupArn': tg_arn,
        },
    ],
    Port = 80,
    Protocol = 'HTTP',
)

In [33]:
lb_url = create_lb_response['LoadBalancers'][0]['DNSName']
lb_url

'Elb3-1698200571.us-east-1.elb.amazonaws.com'

In [34]:
URL = f'http://{lb_url}/admin/login/?next=/admin/'

In [35]:
URL

'http://Elb3-1698200571.us-east-1.elb.amazonaws.com/admin/login/?next=/admin/'

In [47]:
client = requests.session()

client.get(URL) 

csrftoken = client.cookies['csrftoken']

    
login_data = dict(username='cloud', password='cloud', csrfmiddlewaretoken=csrftoken, next='/admin/')
r = client.post(URL, data=login_data, headers=dict(Referer=URL))
cookies = dict(sessionid=client.cookies.get('sessionid'))
print(cookies)

{'sessionid': 'hj1u3ysnuafvmvz8c02f1yxze0fftsez'}


In [55]:
URL_add_user = f"http://{lb_url}/admin/auth/user/add/"
URL_add_group = f"http://{lb_url}/admin/auth/group/add/"

In [57]:
def add_user(URL, username, password1, password2,cookies):
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken, username=username, password1=password1,
                     password2 =password2, _save = "Save")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2
    

In [52]:
def delete_user(user_id, cookies):
    URL_delete = f"http://{lb_url}/admin/auth/user/{user_id}/delete/"
    client = requests.session()
    client.get(URL_delete)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken,post = "yes")
    r2 = client.post(URL_delete, data=post_data, headers=dict(Referer=URL_delete), cookies = cookies)
    return r2

In [60]:
def add_group(URL, groupname ,cookies):
    client = requests.session()
    client.get(URL)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken, name=groupname, _save = "Save")
    r2 = client.post(URL, data=post_data, headers=dict(Referer=URL), cookies = cookies)
    return r2

In [62]:
def delete_group(group_id, cookies):
    URL_delete = f"http://{lb_url}/admin/auth/group/{group_id}/delete/"
    client = requests.session()
    client.get(URL_delete)  # sets cookie
    csrftoken = client.cookies['csrftoken']
    post_data = dict(csrfmiddlewaretoken=csrftoken,post = "yes")
    r2 = client.post(URL_delete, data=post_data, headers=dict(Referer=URL_delete), cookies = cookies)
    return r2

In [58]:
add_user(URL_add_user, 'Joao4', 'Banana888', 'Banana888', cookies)

<Response [200]>

In [59]:
delete_user(4, cookies)

<Response [200]>

In [61]:
add_group(URL_add_group, 'GroupJoao', cookies)

<Response [200]>

In [63]:
delete_group(1, cookies)

<Response [200]>

In [67]:
### DELETE ALL
##OHIO
instance_ohio[0].terminate()
security_group.delete()

{'ResponseMetadata': {'RequestId': '16d81470-30a5-437e-a6fc-198437f469be',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '16d81470-30a5-437e-a6fc-198437f469be',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Tue, 23 Nov 2021 17:21:15 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [70]:
##NORTH VIRGINIA
autoscaling.delete_auto_scaling_group(AutoScalingGroupName ='AutoScalingV1', ForceDelete = True)

{'ResponseMetadata': {'RequestId': '859ca9a7-f1aa-400e-866a-ffb14148f382',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '859ca9a7-f1aa-400e-866a-ffb14148f382',
   'content-type': 'text/xml',
   'content-length': '231',
   'date': 'Tue, 23 Nov 2021 17:25:06 GMT'},
  'RetryAttempts': 0}}

In [72]:
django_img.deregister()

{'ResponseMetadata': {'RequestId': 'e98376ec-e257-4420-9281-51f8976f91c1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e98376ec-e257-4420-9281-51f8976f91c1',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '231',
   'date': 'Tue, 23 Nov 2021 17:26:32 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [76]:
elb.delete_load_balancer(LoadBalancerArn= lb_arn)

{'ResponseMetadata': {'RequestId': '5f0901c7-8cef-48d4-94cd-262bff03f580',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5f0901c7-8cef-48d4-94cd-262bff03f580',
   'content-type': 'text/xml',
   'content-length': '262',
   'date': 'Tue, 23 Nov 2021 17:30:12 GMT'},
  'RetryAttempts': 0}}

In [78]:
elb.delete_target_group(TargetGroupArn= tg_arn)

{'ResponseMetadata': {'RequestId': '51e4ab2b-2716-4579-8c52-f63f613b1762',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '51e4ab2b-2716-4579-8c52-f63f613b1762',
   'content-type': 'text/xml',
   'content-length': '259',
   'date': 'Tue, 23 Nov 2021 17:32:12 GMT'},
  'RetryAttempts': 0}}

In [80]:
autoscaling.delete_launch_configuration(LaunchConfigurationName ='LCV999')

{'ResponseMetadata': {'RequestId': 'a8078b28-1b28-4454-beeb-c5ac312202de',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'a8078b28-1b28-4454-beeb-c5ac312202de',
   'content-type': 'text/xml',
   'content-length': '237',
   'date': 'Tue, 23 Nov 2021 17:33:58 GMT'},
  'RetryAttempts': 0}}

In [81]:
security_group2.delete()

{'ResponseMetadata': {'RequestId': '03166fa3-6678-4911-8244-e4b91bba8e35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '03166fa3-6678-4911-8244-e4b91bba8e35',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '239',
   'date': 'Tue, 23 Nov 2021 17:34:03 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}